In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import re
import unicodedata
import tensorflow_datasets as tfds
import tensorboardcolab as tbc
from tqdm import tqdm_notebook



Using TensorFlow backend.


In [0]:
# tfds.list_builders()

In [3]:
# Will use civil_comments to detect only toxicity from text

ds_train, meta = tfds.load(name = 'imdb_reviews', with_info=True, split=['train', 'test'])

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE1T06W/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE1T06W/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE1T06W/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
train = ds_train[0]
valid = ds_train[1]

In [5]:
for data in train.take(1):
  print(data)

{'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'text': <tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">}


In [6]:
for n, (label, text) in enumerate(train):
  print(data)
  print('--------------')
  # print(data['label'])
  print(label)
  # print(data['text'])
  print(text)
  if n == 0:
    break

{'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'text': <tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">}
--------------
label
text


In [0]:
# Build Tokenizer
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus((data['text'].numpy() for data in train), 
                                                                     target_vocab_size=2**15)

In [8]:
test_sentence = 'this sentence is the test sentence for the tokenizer'

tokenizer.encode(test_sentence)

[14, 10057, 8, 1, 3155, 10057, 22, 1, 330, 9097, 14561, 32508]

In [9]:
max_word_num = len(tokenizer.subwords)

print(max_word_num)

32393


In [10]:
tokenizer.vocab_size

32650

In [11]:
next(iter(train))

{'label': <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 'text': <tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">}

In [0]:
BATCH_SIZE = 64

In [0]:
text_data = train.map(lambda x: x['text'])
label_data = train.map(lambda x: x['label'])

data_zip = tf.data.Dataset.zip((text_data, label_data))

In [14]:
for n, data in enumerate(data_zip):
  print(data)
  # print(label)
  if n == 0:
    break

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [0]:
def text_encode(data, label):
  tokenized_result = tokenizer.encode(data.numpy()) # requires to set bracket!!
  return np.array(tokenized_result), label

In [0]:
def tf_text_encode(data, label):
  result_text, label_ = tf.py_function(text_encode, [data, label], [tf.int64, tf.int64])
  # result_text = tf.cast(result_text, tf.float64)
  label_ = tf.cast(label_, tf.float64)
  return result_text, label_

In [0]:
# def label_encode(text, label):
#   if label >= 0.5:
#     return text, 1.
#   elif label < 0.5 :
#     return text, 0.

In [0]:
# def tf_label_encode(text, label):
#   text, result_label = tf.py_function(label_encode, [text, label], [tf.int32, tf.float64])
#   return text, result_label

In [0]:
dataset = data_zip.map(tf_text_encode)
dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=([None], []))
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [20]:
dataset

<PrefetchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.float64)>

In [21]:
for n, data in enumerate(dataset):
  print(data)
  # print(data[0])
  if n == 0:
    break

(<tf.Tensor: shape=(64, 924), dtype=int64, numpy=
array([[   56,    18,    41, ...,     0,     0,     0],
       [   12,    31,    85, ...,     0,     0,     0],
       [ 8352, 12617,     1, ...,     0,     0,     0],
       ...,
       [  122,  1199,     5, ...,     0,     0,     0],
       [20576,    44,    14, ...,     0,     0,     0],
       [   12,   177, 32433, ...,     0,     0,     0]])>, <tf.Tensor: shape=(64,), dtype=float64, numpy=
array([0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1.])>)


In [22]:
valid

<DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>

In [0]:
valid_text_data = valid.map(lambda x: x['text'])
valid_label_data = valid.map(lambda x: x['label'])
data_zip = tf.data.Dataset.zip((valid_text_data, valid_label_data))
valid_dataset = data_zip.map(tf_text_encode)
valid_dataset = valid_dataset.padded_batch(BATCH_SIZE, padded_shapes=([None], []))
valid_dataset = valid_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [24]:
print(valid_dataset)
print(dataset)

<PrefetchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.float64)>
<PrefetchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.float64)>


In [0]:
tf.random.set_seed(1228)

## None Attention Architecture 

- 1) Embedding -> GRU
- 2) Embedding -> GRU -> GRU
- 3) Embedding -> Conv -> GRU
- 4) Embedding -> GRU -> Conv
- 5) Embedding -> Conv -> GRU -> GRU -> Conv
- 6) Embedding -> Multi-Conv -> Concat -> GRU 

In [0]:
def model():
  input_ = tf.keras.layers.Input(shape=(None,))
  x = tf.keras.layers.Embedding(tokenizer.vocab_size, 300, mask_zero=True)(input_)
  x = tf.keras.layers.GRU(256)(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.models.Model(input_, x)
  return model

In [0]:
model = model()

In [0]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_16 (Embedding)     (None, None, 300)         9795000   
_________________________________________________________________
gru_21 (GRU)                 (None, 256)               428544    
_________________________________________________________________
dense_32 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 129       
Total params: 10,256,569
Trainable params: 10,256,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [0]:
hist_1 = model.fit(dataset, epochs=10, validation_data=valid_dataset)

Epoch 1/10
391/391 [==============================] - 170s 434ms/step - loss: 0.5091 - accuracy: 0.7425 - val_loss: 0.3468 - val_accuracy: 0.8644
Epoch 2/10
391/391 [==============================] - 165s 421ms/step - loss: 0.2008 - accuracy: 0.9243 - val_loss: 0.4270 - val_accuracy: 0.8733
Epoch 3/10
391/391 [==============================] - 165s 423ms/step - loss: 0.0882 - accuracy: 0.9695 - val_loss: 0.4439 - val_accuracy: 0.8563
Epoch 4/10
391/391 [==============================] - 166s 424ms/step - loss: 0.0413 - accuracy: 0.9860 - val_loss: 0.5504 - val_accuracy: 0.8612
Epoch 5/10
391/391 [==============================] - 164s 421ms/step - loss: 0.0310 - accuracy: 0.9884 - val_loss: 0.8294 - val_accuracy: 0.8263
Epoch 6/10
391/391 [==============================] - 165s 423ms/step - loss: 0.0152 - accuracy: 0.9950 - val_loss: 0.7351 - val_accuracy: 0.8626
Epoch 7/10
391/391 [==============================] - 164s 420ms/step - loss: 0.0058 - accuracy: 0.9980 - val_loss: 0.8356 -

## model 1 : loss: 0.0054 - accuracy: 0.9980 - val_loss: 0.9472 - val_accuracy: 0.8604

In [0]:
def model():
  input_ = tf.keras.layers.Input(shape=(None,))
  x = tf.keras.layers.Embedding(tokenizer.vocab_size, 300, mask_zero=True)(input_)
  x = tf.keras.layers.GRU(256, return_sequences=True)(x)
  x = tf.keras.layers.GRU(256)(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.models.Model(input_, x)
  return model

In [0]:
model = model()

In [0]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_17 (Embedding)     (None, None, 300)         9795000   
_________________________________________________________________
gru_22 (GRU)                 (None, None, 256)         428544    
_________________________________________________________________
gru_23 (GRU)                 (None, 256)               394752    
_________________________________________________________________
dense_34 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 129       
Total params: 10,651,321
Trainable params: 10,651,321
Non-trainable params: 0
_______________________________________________

In [0]:
model.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [0]:
hist_2 = model.fit(dataset, epochs=10, validation_data=valid_dataset)

Epoch 1/10
391/391 [==============================] - 280s 716ms/step - loss: 0.4980 - accuracy: 0.7451 - val_loss: 0.3760 - val_accuracy: 0.8422
Epoch 2/10
391/391 [==============================] - 272s 695ms/step - loss: 0.1755 - accuracy: 0.9316 - val_loss: 0.4046 - val_accuracy: 0.8683
Epoch 3/10
391/391 [==============================] - 272s 696ms/step - loss: 0.0898 - accuracy: 0.9667 - val_loss: 0.5058 - val_accuracy: 0.8635
Epoch 4/10
391/391 [==============================] - 272s 696ms/step - loss: 0.0505 - accuracy: 0.9822 - val_loss: 0.6424 - val_accuracy: 0.8497
Epoch 5/10
391/391 [==============================] - 272s 697ms/step - loss: 0.0271 - accuracy: 0.9910 - val_loss: 0.6965 - val_accuracy: 0.8602
Epoch 6/10
391/391 [==============================] - 272s 696ms/step - loss: 0.0191 - accuracy: 0.9933 - val_loss: 0.6745 - val_accuracy: 0.8622
Epoch 7/10
391/391 [==============================] - 273s 698ms/step - loss: 0.0221 - accuracy: 0.9926 - val_loss: 0.6891 -

## Model 2 loss: 0.0151 - accuracy: 0.9953 - val_loss: 0.7596 - val_accuracy: 0.8623

In [0]:
def model():
  input_ = tf.keras.layers.Input(shape=(None,))
  x = tf.keras.layers.Embedding(tokenizer.vocab_size, 300, mask_zero=True)(input_)
  x = tf.keras.layers.Conv1D(258, 3, 1, 'valid', activation='relu')(x)
  x = tf.keras.layers.GRU(256)(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.models.Model(input_, x)
  return model

In [0]:
model = model()

In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 300)         9795000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 258)         232458    
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               396288    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 10,456,771
Trainable params: 10,456,771
Non-trainable params: 0
_______________________________________________

In [0]:
model.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [0]:
hist_3 = model.fit(dataset, epochs=100, validation_data=valid_dataset)

Epoch 1/100
391/391 [==============================] - 342s 876ms/step - loss: 0.6938 - accuracy: 0.4986 - val_loss: 0.6930 - val_accuracy: 0.5005
Epoch 2/100
391/391 [==============================] - 218s 558ms/step - loss: 0.6917 - accuracy: 0.5026 - val_loss: 0.6937 - val_accuracy: 0.4992
Epoch 3/100
391/391 [==============================] - 219s 560ms/step - loss: 0.6905 - accuracy: 0.4961 - val_loss: 0.6993 - val_accuracy: 0.4989
Epoch 4/100
391/391 [==============================] - 218s 559ms/step - loss: 0.6847 - accuracy: 0.4994 - val_loss: 0.7032 - val_accuracy: 0.5005
Epoch 5/100
391/391 [==============================] - 219s 559ms/step - loss: 0.6835 - accuracy: 0.5069 - val_loss: 0.6988 - val_accuracy: 0.5004
Epoch 6/100
391/391 [==============================] - 219s 561ms/step - loss: 0.6829 - accuracy: 0.5053 - val_loss: 0.7036 - val_accuracy: 0.5024
Epoch 7/100
391/391 [==============================] - 219s 561ms/step - loss: 0.6827 - accuracy: 0.5019 - val_loss: 0

KeyboardInterrupt: ignored

## Model 3 loss: 1.1065e-08 - accuracy: 1.0000 - val_loss: 1.8439 - val_accuracy: 0.8592

In [0]:
def model():
  input_ = tf.keras.layers.Input(shape=(None,))
  x = tf.keras.layers.Embedding(tokenizer.vocab_size, 300, mask_zero=True)(input_)
  x = tf.keras.layers.GRU(256, return_sequences=True)(x)
  x = tf.keras.layers.Conv1D(258, 3, 1, 'valid', activation='relu')(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.models.Model(input_, x)
  return model

In [0]:
model = model()

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 300)         9795000   
_________________________________________________________________
gru (GRU)                    (None, None, 256)         428544    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 258)         198402    
_________________________________________________________________
dense (Dense)                (None, None, 128)         33152     
_________________________________________________________________
dense_1 (Dense)              (None, None, 1)           129       
Total params: 10,455,227
Trainable params: 10,455,227
Non-trainable params: 0
_________________________________________________

In [0]:
model.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [0]:
hist_4 = model.fit(dataset, epochs=100, validation_data=valid_dataset)

Epoch 1/100
391/391 [==============================] - 127s 325ms/step - loss: 0.6622 - accuracy: 0.5449 - val_loss: 0.6556 - val_accuracy: 0.5604
Epoch 2/100
391/391 [==============================] - 97s 248ms/step - loss: 0.6006 - accuracy: 0.5945 - val_loss: 0.6326 - val_accuracy: 0.5776
Epoch 3/100
391/391 [==============================] - 97s 248ms/step - loss: 0.5676 - accuracy: 0.6077 - val_loss: 0.6579 - val_accuracy: 0.5736
Epoch 4/100
391/391 [==============================] - 98s 250ms/step - loss: 0.5483 - accuracy: 0.6146 - val_loss: 0.7551 - val_accuracy: 0.5619
Epoch 5/100
391/391 [==============================] - 97s 249ms/step - loss: 0.5549 - accuracy: 0.6151 - val_loss: 0.7017 - val_accuracy: 0.5591
Epoch 6/100
391/391 [==============================] - 97s 248ms/step - loss: 0.5410 - accuracy: 0.6198 - val_loss: 0.7517 - val_accuracy: 0.5660
Epoch 7/100
391/391 [==============================] - 97s 249ms/step - loss: 0.5275 - accuracy: 0.6246 - val_loss: 0.8589 

## Model4 loss: 0.5100 - accuracy: 0.6302 - val_loss: 1.7039 - val_accuracy: 0.5392

In [0]:
def model():
  input_ = tf.keras.layers.Input(shape=(None,))
  x = tf.keras.layers.Embedding(tokenizer.vocab_size, 300, mask_zero=True)(input_)
  x = tf.keras.layers.Conv1D(258, 3, 1, 'valid', activation='relu')(x)
  x = tf.keras.layers.GRU(256, return_sequences=True)(x)
  x = tf.keras.layers.GRU(256, return_sequences=True)(x)
  x = tf.keras.layers.Conv1D(258, 3, 1, 'valid', activation='relu')(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.models.Model(input_, x)
  return model

In [0]:
model = model()

In [34]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 300)         9795000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 258)         232458    
_________________________________________________________________
gru (GRU)                    (None, None, 256)         396288    
_________________________________________________________________
gru_1 (GRU)                  (None, None, 256)         394752    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 258)         198402    
_________________________________________________________________
dense (Dense)                (None, None, 128)         33152 

In [0]:
model.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [37]:
hist_5 = model.fit(dataset, epochs=100, validation_data=valid_dataset)

Epoch 1/100
391/391 [==============================] - 606s 2s/step - loss: 0.5509 - accuracy: 0.6750 - val_loss: 0.3931 - val_accuracy: 0.8306
Epoch 2/100
391/391 [==============================] - 400s 1s/step - loss: 0.2233 - accuracy: 0.9111 - val_loss: 0.4029 - val_accuracy: 0.8524
Epoch 3/100
391/391 [==============================] - 401s 1s/step - loss: 0.1205 - accuracy: 0.9551 - val_loss: 0.4664 - val_accuracy: 0.8403
Epoch 4/100
391/391 [==============================] - 399s 1s/step - loss: 0.0818 - accuracy: 0.9689 - val_loss: 0.5666 - val_accuracy: 0.8251
Epoch 5/100
391/391 [==============================] - 398s 1s/step - loss: 0.0571 - accuracy: 0.9785 - val_loss: 0.6783 - val_accuracy: 0.8203
Epoch 6/100
391/391 [==============================] - 399s 1s/step - loss: 0.0508 - accuracy: 0.9803 - val_loss: 0.8597 - val_accuracy: 0.8045
Epoch 7/100
391/391 [==============================] - 399s 1s/step - loss: 0.0443 - accuracy: 0.9832 - val_loss: 0.8410 - val_accuracy:

KeyboardInterrupt: ignored

## Model 5 loss: 0.0571 - accuracy: 0.9785 - val_loss: 0.6783 - val_accuracy: 0.8203

In [0]:
def model():
  input_ = tf.keras.layers.Input(shape=(None,))
  x = tf.keras.layers.Embedding(tokenizer.vocab_size, 300, mask_zero=True)(input_)

  conv_layers = []
  for size in range(1, 7):
    x = tf.keras.layers.Conv1D(258, size, 1, 'causal', activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    conv_layers.append(x)
  concat = tf.keras.layers.Concatenate()(conv_layers)
  x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(128, activation='relu'))(concat)
  x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.models.Model(input_, x)
  return model

In [0]:
model = model()

In [66]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, None, 300)    9795000     input_11[0][0]                   
__________________________________________________________________________________________________
conv1d_116 (Conv1D)             (None, None, 258)    77658       embedding_10[0][0]               
__________________________________________________________________________________________________
batch_normalization_112 (BatchN (None, None, 258)    1032        conv1d_116[0][0]                 
____________________________________________________________________________________________

In [0]:
model.compile(loss = 'binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [0]:
hist_6 = model.fit(dataset, epochs=100, validation_data=valid_dataset)

Epoch 1/100
391/391 [==============================] - 952s 2s/step - loss: 0.7415 - accuracy: 0.5321 - val_loss: 0.6868 - val_accuracy: 0.5347
Epoch 2/100
391/391 [==============================] - 363s 928ms/step - loss: 0.6758 - accuracy: 0.5559 - val_loss: 0.6733 - val_accuracy: 0.5425
Epoch 3/100
391/391 [==============================] - 362s 926ms/step - loss: 0.6565 - accuracy: 0.5651 - val_loss: 0.6792 - val_accuracy: 0.5399
Epoch 4/100
391/391 [==============================] - 363s 927ms/step - loss: 0.6371 - accuracy: 0.5805 - val_loss: 0.6912 - val_accuracy: 0.5371
Epoch 5/100
391/391 [==============================] - 361s 923ms/step - loss: 0.6146 - accuracy: 0.5863 - val_loss: 0.7497 - val_accuracy: 0.5337
Epoch 6/100
391/391 [==============================] - 360s 921ms/step - loss: 0.5978 - accuracy: 0.5939 - val_loss: 0.8231 - val_accuracy: 0.5327
Epoch 7/100
391/391 [==============================] - 357s 913ms/step - loss: 0.5863 - accuracy: 0.5976 - val_loss: 0.89